# Build your Own Model

In [1]:
import pandas as pd
import numpy as np
import torch
import pyro
import pyro.distributions as dist

import orbit
from orbit.models.template import BaseTemplate
from orbit.models.template import FullBayesianTemplate
from orbit.estimators.pyro_estimator import PyroEstimatorVI

from orbit.utils.simulation import make_regression

%matplotlib inline

In [2]:
assert orbit.__version__ == '1.0.13dev'

In [3]:
class Fitter:
    max_plate_nesting = 1  # max number of plates nested in model

    def __init__(self, data):
        for key, value in data.items():
            key = key.lower()
            if isinstance(value, (list, np.ndarray)):
                value = torch.tensor(value, dtype=torch.float)
            self.__dict__[key] = value

    def __call__(self):
        extra_out = {}
        
        p = self.regressor.shape[1]
        bias = pyro.sample("bias", dist.Normal(0, 1))
        weight = pyro.sample("weight", dist.Normal(0, 1).expand([p]).to_event(1))
        yhat = bias + weight @ self.regressor.transpose(-1, -2)
        obs_sigma = pyro.sample("obs_sigma", dist.HalfCauchy(self.response_sd))
        
        with pyro.plate("response_plate", self.num_of_obs):
            pyro.sample("response", dist.Normal(yhat, obs_sigma), obs=self.response)
        return extra_out

In [12]:
from enum import Enum

class MyDataMapper(Enum):
    NUM_OF_OBSERVATIONS = 'NUM_OF_OBS'
    RESPONSE = 'RESPONSE'
    RESPONSE_SD = 'RESPONSE_SD'
    REGRESSOR = 'REGRESSOR'

In [13]:
class BaseRegression(BaseTemplate):
    _fitter = Fitter
    _data_input_mapper = MyDataMapper
    def __init__(self, regressor_col, **kwargs):
        super().__init__(**kwargs)  # create estimator in base class
        self.regressor_col = regressor_col
        self.regressor = None

    def _set_model_param_names(self):
        self._model_param_names = ['bias', 'weight', 'obs_sigma']
        
    def _set_dynamic_attributes(self, df):
        super()._validate_training_df(df)
        super()._set_training_df_meta(df)
        
        self.regressor = df[self.regressor_col].values

        super()._set_model_data_input()
        self._set_init_values()
        

class PyroVIRegression(FullBayesianTemplate, BaseRegression):
    
    _supported_estimator_types = [PyroEstimatorVI]
    
    def __init__(self, estimator_type=PyroEstimatorVI, **kwargs):
        super().__init__(estimator_type=estimator_type, **kwargs)

In [22]:
x, y, coefs = make_regression(100, [3.0, -1.0], bias=1.0)

In [23]:
df = pd.DataFrame(
    np.concatenate([y.reshape(-1, 1), x], axis=1), columns=['y', 'x1', 'x2']
)
df['week'] = pd.date_range(start='2016-01-04', periods=len(y), freq='7D')

In [24]:
df.head(5)

,y,x1,x2,week
0,1.863960,0.172792,0.000000,2016-01-04
1,2.317274,0.165219,-0.000000,2016-01-11
2,2.465284,0.452678,0.223187,2016-01-18
3,-0.593716,-0.000000,0.290559,2016-01-25
4,2.305148,0.182286,0.147066,2016-02-01


In [25]:
mod = PyroVIRegression(
    response_col='y', 
    date_col='week',
    regressor_col=['x1','x2'], 
    verbose=True,
    num_steps=501)

In [26]:
mod.fit(df)

INFO:root:Guessed max_plate_nesting = 2


step    0 loss = 23912, scale = 0.090377
step  100 loss = 12587, scale = 0.016337
step  200 loss = 12596, scale = 0.0164
step  300 loss = 12588, scale = 0.016334
step  400 loss = 12591, scale = 0.015944
step  500 loss = 12588, scale = 0.016318


In [27]:
mod.response_sd

1.5622762644474886

In [28]:
estimated_weights = mod._posterior_samples['weight']

In [29]:
np.median(estimated_weights, axis=0)

array([ 2.9027781, -0.9494698], dtype=float32)